# Inilization

In [1]:
#Sedona Imports
import sedona.sql
from sedona.register import SedonaRegistrator
from sedona.utils import SedonaKryoRegistrator, KryoSerializer
from sedona.core.SpatialRDD import PolygonRDD, PointRDD
from sedona.core.enums import FileDataSplitter
import pyspark.sql.types as pst
from pyspark import StorageLevel
from pyspark.sql import SparkSession 

In [3]:
spark = SparkSession.builder \
    .appName('Vessel_Traffic_Indonesia') \
    .config("spark.serializer", KryoSerializer.getName) \
    .config("spark.kryo.registrator", SedonaKryoRegistrator.getName) \
    .config("spark.sql.parquet.enableVectorizedReader", "false") \
    .config('spark.jars.packages', 'org.apache.sedona:sedona-python-adapter-3.0_2.12:1.0.1-incubating,org.apache.sedona:sedona-viz-3.0_2.12:1.0.1-incubating') \
    .getOrCreate()

SedonaRegistrator.registerAll(spark)

True

In [4]:
#For 3.3.2
from shapely.geometry import Point, Polygon, mapping
import h3.api.numpy_int as h3int

In [5]:
import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.functions import col, when, date_format, count

In [6]:
basepath = "s3a://ungp-ais-data-historical-backup/exact-earth-data/transformed/prod/"

In [7]:
# Path
save_path = "s3a://ungp-ais-data-historical-backup/user_temp/"
path_unique = save_path + "222011349/"

# Data AIS

In [8]:
# Read Data
data_ais = spark.read.parquet(path_unique + "data-ais-ihs-indonesia-by-mmsi-filter-2022.parquet", header=True)

In [7]:
data_ais.count()

1696875705

# Filter Data

In [9]:
# Filter Record < 10

# Menghitung jumlah record per MMSI
record_counts = data_ais.groupBy("mmsi").agg(count("*").alias("record_count"))

# Mendapatkan MMSI dengan record kurang dari 10
mmsi_less_than_10 = record_counts.filter(col("record_count") < 10).select("mmsi")

# Menghapus MMSI dengan record kurang dari 10 dari DataFrame asli menggunakan left_anti join
mmsi_with_record_great_10 = data_ais.join(mmsi_less_than_10, on="mmsi", how="left_anti")

In [17]:
mmsi_with_record_great_10.count()

1444392908

In [10]:
# Filter SOG > 3 berjumlah < 20 

# Filter data berdasarkan kondisi SOG lebih dari 3
filtered_data = mmsi_with_record_great_10.filter(col("sog") > 3)

# Kelompokkan data berdasarkan MMSI dan hitung jumlah catatan
grouped_data = filtered_data.groupBy("mmsi").agg(count("*").alias("record_count"))

# Filter MMSI yang memiliki SOG lebih dari 3 tetapi kurang dari 20
filtered_mmsi = grouped_data.filter((col("record_count") < 20)).select("mmsi")

# Hapus baris yang terkait dengan MMSI yang telah difilter dari DataFrame
mmsi_with_sog_greater3_greater20 = mmsi_with_record_great_10.join(filtered_mmsi, "mmsi", "left_anti")

In [ ]:
mmsi_with_sog_greater3_greater20.count()

1443592577

## Save Data

In [11]:
data_ais_filtered = mmsi_with_sog_greater3_greater20

In [ ]:
# Save Data
data_ais_filtered.write.option("header", True).mode("overwrite").parquet(path_unique + "data-ais-ihs-indonesia-by-mmsi-filter-2022-v2.parquet")

Py4JJavaError: An error occurred while calling o195.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:638)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:278)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:98)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$6(SQLExecution.scala:109)
	at org

In [28]:
spark.stop()